In [36]:
import random

# Global counters
swap_count = 0
comparison_count = 0
pivot_cost_total = 0  # Track total pivot selection cost

# Swap function with tracking
def swap(arr, i, j):
    global swap_count
    if i != j:
        arr[i], arr[j] = arr[j], arr[i]
        swap_count += 1

# Cost function for pivot selection efficiency
# Lower cost = better pivot. Same as the Simulated Annealing version.
def pivot_cost(arr, low, high, pivot_index):
    pivot_value = arr[pivot_index]
    left = []
    right = []
    comparisons = 0
    swaps = 0  # Simulated; can be improved later

    for i in range(low, high + 1):
        if i == pivot_index:
            continue
        comparisons += 1
        if arr[i] < pivot_value:
            left.append(arr[i])
        else:
            right.append(arr[i])

    total_size = len(left) + len(right)
    balance_factor = abs(len(left) - len(right)) / total_size if total_size > 0 else 1

    return comparisons + swaps + (balance_factor * total_size)

# Partition function with selectable pivot strategy
def partition(arr, low, high, pivot_type="last"):
    global comparison_count, pivot_cost_total

    # Choose pivot based on strategy 
    if pivot_type == "first":
        pivot_index = low
    elif pivot_type == "random":
        pivot_index = random.randint(low, high)
    elif pivot_type == "median":
        middle = (low + high) // 2
        # Median-of-three pivot strategy
        pivot_index = sorted([(low, arr[low]), (middle, arr[middle]), (high, arr[high])], key=lambda x: x[1])[1][0]
    else:  # Default: "last"
        pivot_index = high

    # Compute pivot selection cost (for evaluation only)
    cost = pivot_cost(arr, low, high, pivot_index)
    pivot_cost_total += cost

    # Move chosen pivot to end for partitioning
    swap(arr, pivot_index, high)
    pivot = arr[high]

    # Index of smaller element and indicates
    # the right position of pivot found so far
    i = low - 1

    # Traverse arr[low..high] and move all smaller elements 
    # to the left side. Elements from low to i are smaller 
    # after every iteration
    for j in range(low, high):
        comparison_count += 1  # Count comparisons
        if arr[j] < pivot:
            i += 1
            swap(arr, i, j)

    # Move pivot after smaller elements and return its position
    swap(arr, i + 1, high)
    return i + 1

# Main QuickSort recursive function
def quickSort(arr, low, high, pivot_type="last"):
    if low < high:
        # pi is the partition return of the index of pivot
        pi = partition(arr, low, high, pivot_type)

        # Recursion calls for smaller elements and greater or equal elements
        quickSort(arr, low, pi - 1, pivot_type)
        quickSort(arr, pi + 1, high, pivot_type)

# Function to reset counters and run QuickSort
def run_quick_sort(arr, pivot_type):
    global swap_count, comparison_count, pivot_cost_total
    swap_count = 0
    comparison_count = 0
    pivot_cost_total = 0
    arr_copy = arr[:]  # Create a copy to not modify the original array
    quickSort(arr_copy, 0, len(arr_copy) - 1, pivot_type)
    return arr_copy, swap_count, comparison_count, pivot_cost_total

# Example usage
if __name__ == "__main__":
    arr = [0,1,2] # Sample input array

    for pivot_strategy in ["last", "first", "random", "median"]:
        sorted_arr, swaps, comparisons, total_pivot_cost = run_quick_sort(arr, pivot_strategy)
        print(f"\nPivot Strategy: {pivot_strategy}")
        print("Original Array: ", arr)
        print("Sorted Array:   ", sorted_arr)
        print(f"Total Swaps: {swaps}")
        print(f"Total Comparisons: {comparisons}")
        print(f"Total Pivot Selection Cost: {total_pivot_cost:.4f}")



Pivot Strategy: last
Original Array:  [0, 1, 2]
Sorted Array:    [0, 1, 2]
Total Swaps: 0
Total Comparisons: 3
Total Pivot Selection Cost: 6.0000

Pivot Strategy: first
Original Array:  [0, 1, 2]
Sorted Array:    [0, 1, 2]
Total Swaps: 4
Total Comparisons: 3
Total Pivot Selection Cost: 6.0000

Pivot Strategy: random
Original Array:  [0, 1, 2]
Sorted Array:    [0, 1, 2]
Total Swaps: 2
Total Comparisons: 2
Total Pivot Selection Cost: 2.0000

Pivot Strategy: median
Original Array:  [0, 1, 2]
Sorted Array:    [0, 1, 2]
Total Swaps: 2
Total Comparisons: 2
Total Pivot Selection Cost: 2.0000


In [4]:
# 2. Implementing Simulated Annealing
import math
import random

# Objective function: Rastrigin function
def objective_function(x):
    return 10 * len(x) + sum([xi**2 - 10 * math.cos(2 * math.pi * xi) for xi in x])

# Creating the Neighbor function 
def get_neighbor(x, step_size=0.1):
    neighbor = x[:]
    index = random.randint(0, len(x) - 1) # Return a random integer from 0 to the length of x.
    neighbor[index] += random.uniform(-step_size, step_size) # Returning a random float value between the beginning and end of step_size
    return neighbor

# Creating the simulated annealing function
# We will initialize the solution, update the temp, generate new candidates, and decide whether to accept them
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
    best = [random.uniform(bound[0], bound[1]) for bound in bounds]
    best_eval = objective(best)
    current, current_eval = best, best_eval
    scores = [best_eval]

    for i in range(n_iterations):
        # Decreasing temperature
        t = temp / float(i + 1)
        
        # Generate a candidate solution
        candidate = get_neighbor(current, step_size)
        candidate_eval = objective(candidate)
        
        # Checking if we can keep the solution
        if candidate_eval < best_eval or random.random() < math.exp((current_eval - candidate_eval) / t):
            current, current_eval = candidate, candidate_eval
            if candidate_eval < best_eval:
                best, best_eval = candidate, candidate_eval
                scores.append(best_eval)

        if i % 100 == 0:
            print(f"Iteration {i}, Temperature {t:.3f}, Best Evaluation {best_eval:.5f}")
    return best, best_eval, scores


# Defining the problem domain
bounds = [(-5.0, 5.0) for _ in range(2)]
n_iterations = 1000
step_size = 0.1
temp = 10

# Persorming simulated annealing search
best, score, scores = simulated_annealing(objective_function, bounds, n_iterations, step_size, temp)

print(f'Best Solution: {best}')
print(f'Best Score: {score}')

Iteration 0, Temperature 10.000, Best Evaluation 50.67021
Iteration 100, Temperature 0.099, Best Evaluation 49.74787
Iteration 200, Temperature 0.050, Best Evaluation 49.74787
Iteration 300, Temperature 0.033, Best Evaluation 49.74787
Iteration 400, Temperature 0.025, Best Evaluation 49.74787
Iteration 500, Temperature 0.020, Best Evaluation 49.74787
Iteration 600, Temperature 0.017, Best Evaluation 49.74787
Iteration 700, Temperature 0.014, Best Evaluation 49.74787
Iteration 800, Temperature 0.012, Best Evaluation 49.74787
Iteration 900, Temperature 0.011, Best Evaluation 49.74787
Best Solution: [-4.97351998724881, 4.97379330463322]
Best Score: 49.74787253457849


In [47]:
import random
import math

# Global counters
swap_count = 0
comparison_count = 0
pivot_cost_total = 0  # Accumulate total pivot cost

# Swap function: exchanges elements and tracks swap count
def swap(arr, i, j):
    global swap_count
    if i != j:
        arr[i], arr[j] = arr[j], arr[i]
        swap_count += 1

# Partition function using a given pivot index
def partition(arr, low, high, pivot_index):
    global comparison_count
    swap(arr, pivot_index, high)  # Move pivot to end
    pivot = arr[high]
    i = low - 1
    for j in range(low, high):
        comparison_count += 1
        if arr[j] < pivot:
            i += 1
            swap(arr, i, j)
    swap(arr, i + 1, high)
    return i + 1

# Pivot cost function: measures how balanced a pivot splits the array
def pivot_cost(arr, low, high, pivot_index):
    pivot_value = arr[pivot_index]
    left = []
    right = []
    comparisons = 0

    for i in range(low, high + 1):
        if i == pivot_index:
            continue
        comparisons += 1
        if arr[i] < pivot_value:
            left.append(arr[i])
        else:
            right.append(arr[i])
    
    total_size = len(left) + len(right)
    balance_factor = abs(len(left) - len(right)) / total_size if total_size > 0 else 1

    # Return cost as a weighted sum of comparisons and imbalance
    return comparisons + (balance_factor * total_size)

# Simulated Annealing to find an optimal pivot index for a subarray
def simulated_annealing_pivot(arr, low, high, n_iterations=100, initial_temp=10):
    global pivot_cost_total

    # Initialize with a random pivot index
    current = random.randint(low, high)
    best = current
    current_cost = pivot_cost(arr, low, high, current)
    best_cost = current_cost
    temp = initial_temp

    print(f"\n--- Simulated Annealing for Subarray [{low}:{high}] ---")
    print(f"Initial Pivot: Index={current}, Value={arr[current]}, Cost={current_cost:.4f}")

    for i in range(n_iterations):
        temperature = temp / float(i + 1)

        # Generate a neighboring pivot index
        if random.random() < 0.8:
            neighbor = current + random.choice([-1, 1])
            neighbor = max(low, min(high, neighbor))  # Keep in bounds
        else:
            neighbor = random.randint(low, high)

        neighbor_cost = pivot_cost(arr, low, high, neighbor)
        delta = neighbor_cost - current_cost

        # Accept better or probabilistically worse solution
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            current = neighbor
            current_cost = neighbor_cost
            if neighbor_cost < best_cost:
                best = neighbor
                best_cost = neighbor_cost

        # Display progress every 10 iterations
        if i % 10 == 0 or i == n_iterations - 1:
            print(f"Iteration {i:3d} | Temp: {temperature:6.3f} | "
                  f"Current: idx={current}, value={arr[current]}, cost={current_cost:.4f} | "
                  f"Best: idx={best}, value={arr[best]}, cost={best_cost:.4f}")

    print(f"Selected Pivot: Index={best}, Value={arr[best]}, Cost={best_cost:.4f}")
    print("-" * 60)

    pivot_cost_total += best_cost  # Track total cost
    return best

# QuickSort using Simulated Annealing to choose pivots
def quickSort(arr, low, high):
    if low < high:
        pivot_index = simulated_annealing_pivot(arr, low, high)
        pi = partition(arr, low, high, pivot_index)
        quickSort(arr, low, pi - 1)
        quickSort(arr, pi + 1, high)

# Wrapper to run and track performance metrics
def run_quick_sort_with_sa(arr):
    global swap_count, comparison_count, pivot_cost_total
    swap_count = 0
    comparison_count = 0
    pivot_cost_total = 0
    arr_copy = arr[:]
    quickSort(arr_copy, 0, len(arr_copy) - 1)
    return arr_copy, swap_count, comparison_count, pivot_cost_total

# Example usage
if __name__ == "__main__":
    arr = [0,1,2]
    sorted_arr, swaps, comparisons, total_pivot_cost = run_quick_sort_with_sa(arr)

    print("\nSorted Array:", sorted_arr)
    print(f"Total Swaps: {swaps}")
    print(f"Total Comparisons: {comparisons}")
    print(f"Total Pivot Selection Cost: {total_pivot_cost:.4f}")



--- Simulated Annealing for Subarray [0:2] ---
Initial Pivot: Index=0, Value=0, Cost=4.0000
Iteration   0 | Temp: 10.000 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  10 | Temp:  0.909 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  20 | Temp:  0.476 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  30 | Temp:  0.323 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  40 | Temp:  0.244 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  50 | Temp:  0.196 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  60 | Temp:  0.164 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  70 | Temp:  0.141 | Current: idx=1, value=1, cost=2.0000 | Best: idx=1, value=1, cost=2.0000
Iteration  80 | Temp:  0.123 | Current: idx=1, value=1, cost=2.0000 | Best: